In [1]:
import pandas as pd
import numpy as np
import random
import pickle

from numpy import exp, sqrt, dot
from scipy.spatial.distance import cdist
from scipy.io import loadmat
from random import shuffle

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

from scipy.stats import kurtosis
from scipy.stats import moment
from scipy.stats import skew

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset = "MODIS"

In [3]:
def load_data():
    
    if dataset == "CORN" or dataset == "WHEAT":
        full_data = pickle.load(open(dataset + ".p", "rb"))
        columns = (["id"] +  ['label'] + ["reflectance_" + str(i) for i in range(92)])
        full_data.columns = columns
        
    else:
        mat_dict = loadmat('MIR/' + dataset + '.mat')
        full_data = pd.DataFrame(mat_dict[dataset])

        # Rename columns to something more interpretable
        columns = (["id"] + ["reflectance_" + str(i) for i in range(7)]
                   + ["solar_" + str(i) for i in range(5)] + ['label'])
        full_data.columns = columns

        if dataset == "MISR2":
            full_data = full_data[full_data['id'].isin(list(full_data['id'].value_counts().index[full_data['id'].value_counts() == 100]))]

    return full_data

In [4]:
def moments_df(n_moments):
    
    full_data = load_data()

#     non_constant_cols = ['id', 'reflectance_0', 'reflectance_1', 'reflectance_2',
#            'reflectance_3', 'reflectance_4', 'reflectance_5', 'reflectance_6']
    
    non_constant_cols = (["id"] + ["reflectance_" + str(i) for i in range(12)])

    constant_columns = ['solar_0', 'solar_1', 'solar_2', 'solar_3']
    list_operation = ['np.mean', 'np.var', 'skew', 'kurtosis']

    full_data_subset = full_data[non_constant_cols]
    
    dic_df = {}

    for moments in range(1, n_moments+1):
        if moments < 5:
            dic_df[moments] = eval("np.stack(full_data_subset.groupby('id').apply(lambda group: " + 
                                   list_operation[moments-1] + "(group)).values)[:,1:]")
        else:
            dic_df[moments] = np.stack(full_data_subset.groupby('id').apply(lambda group: moment(group, moment=moments)))[:,1:]

    array_data = dic_df[1]
    
    if n_moments > 1:
        for key in range(2, n_moments+1):
            array_data = np.hstack([array_data, dic_df[key]])   

    df = pd.DataFrame(array_data)
    if dataset != "CORN" and dataset != "WHEAT":
        df = pd.concat([df, full_data.groupby('id').mean()[constant_columns].reset_index()], axis=1)
    df['id'] = full_data['id'].unique()
    df['label'] = full_data.groupby('id').mean()['label'].values
    df = df.set_index('id')
    
    col_label = ['label']
    labels = df[col_label]
    df = df[[col for col in df.columns if col not in col_label]]
    
    return df, labels

In [5]:
def make_sequences(train, test):
    list_sequence_train, list_sequence_test = [], []
    list_train_y, list_test_y = [], []
    features = [col for col in train.columns if col not in ['id', 'label']]
    
    for bag_id in list(train['id'].unique()):
        list_sequence_train.append(np.array(train[train['id']==bag_id][features]))
        list_train_y.append(np.array(train[train['id']==bag_id]['label'])[0])
        
    for bag_id in list(test['id'].unique()):
        list_sequence_test.append(np.array(test[test['id']==bag_id][features]))
        list_test_y.append(np.array(test[test['id']==bag_id]['label'])[0])
    
    return list_sequence_train, list_sequence_test, list_train_y, list_test_y

In [6]:
def inference_attention(random_seed, splits=5, use_moments=False, n_moments=1):

    full_data = load_data()
    num_features = 12
    count = 1
    
    if use_moments:
        num_features = 92 + n_moments*92
        df, _ = moments_df(n_moments)
        col_solar = ['solar_0', 'solar_1', 'solar_2', 'solar_3', 'solar_4']
        df = df.iloc[np.repeat(np.arange(len(df)), 100)]
        df = df[[col for col in df.columns if col not in col_solar]]
        df = df.reset_index(drop=True)
        full_data = pd.concat([full_data, df], axis=1)
    
    kf = KFold(n_splits=splits, shuffle=True, random_state=random_seed)
    cols_exclude = ["id", "label"]
    features = [col for col in list(full_data.columns) if col not in cols_exclude]
    lowest_val_list, loss_test_list = [], []
    
    for train_index, test_index in kf.split(list(full_data['id'].unique())):
        #print('Compute for FOLD NUMBER ' + str(count))
        train_index = full_data['id'].unique()[train_index]
        test_index = full_data['id'].unique()[test_index]
        train = full_data[full_data['id'].apply(lambda value: value in train_index)]
        test = full_data[full_data['id'].apply(lambda value: value in test_index)]
        
        scaler = StandardScaler()
        scaler.fit(train[features])
        train[features], test[features] = scaler.transform(train[features]), scaler.transform(test[features])
    
        list_sequence_train, list_sequence_test, train_y, test_y = make_sequences(train, test)
        lowest_val_loss, loss_test = transformer(list_sequence_train, list_sequence_test, train_y, test_y, 
                                               num_features)
        
        lowest_val_list.append(lowest_val_loss)
        loss_test_list.append(loss_test)
        
        tf.reset_default_graph()
        count += 1
    
    return lowest_val_loss, loss_test_list

In [7]:
def compute_rmse(pred_array, truth_array):
    loss = np.sqrt(mean_squared_error(np.reshape(np.array(pred_array), (np.array(pred_array).shape[0],1)), 
                                               np.reshape(np.array(truth_array), (np.array(truth_array).shape[0],1))))
    return loss

In [8]:
# check the neighbours of the minimum to be sure it is not due to stochasticity
def get_min_index(array, n_max=3):
    indices = array.argsort()[:n_max] 
    
    avg_neighbourgs = []
    for val in indices:
        if val == len(array)-1:
            avg_neighbourgs.append((array[val]+array[val-1]+array[val-2])/3)
        else:
            avg_neighbourgs.append((array[val]+array[val-1]+array[val+1])/3)
    
    index = indices[np.argmin(avg_neighbourgs)]
    
    return index

# Transformer Network

In [56]:
num_features = 12
sequence_lenght = 100
learning_rate = 0.00005
n_epochs = 600
num_test_runs = 1
dropout_input = 1.0
dropout_output = 0.7

dim_transform1 = 16
head_number = 1
layer_number = 3

In [57]:
def normalize_with_moments(x, axes=0, epsilon=1e-8):
    mean, variance = tf.nn.moments(x, axes=axes)
    x_normed = (x - mean) / tf.sqrt(variance + epsilon) # epsilon to avoid dividing by zero
    return x_normed

In [58]:
def self_attention(q, k, v):
    
    matmul_qk = tf.matmul(q, k, transpose_b=True) 
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1) 

    output = tf.matmul(attention_weights, v)

    return output, attention_weights

In [59]:
def multi_head_layer(X_or, layer_number, head_number, prob_output, multi_head_concat, num_features):
    
    with tf.variable_scope('attention_head_'+ str(layer_number) + '_' + str(head_number)):
            
            if layer_number > 1:
                num_features = dim_transform1
                
            W_K = tf.Variable(tf.random_normal(shape = (num_features, dim_transform1), stddev = 0.1), name = "weights_keys", trainable = True)
            W_Q = tf.Variable(tf.random_normal(shape = (num_features, dim_transform1), stddev = 0.1), name = "weights_queries", trainable = True)
            W_V = tf.Variable(tf.random_normal(shape = (num_features, dim_transform1), stddev = 0.1), name = "weights_values", trainable = True)

            queries = tf.matmul(X_or, W_K) # try with relu on these after that run finishes
            keys = tf.matmul(X_or, W_Q)
            values = tf.matmul(X_or, W_V)

            output, attention_weights = self_attention(queries, keys, values)
            output = tf.nn.dropout(output, prob_output, noise_shape=[1,dim_transform1])
            #output = tf.add(values, output) # residual layer
            #output = normalize_with_moments(output) # normalize residuals
            multi_head_concat = tf.concat([multi_head_concat, output], axis = 1)
    
    return multi_head_concat

In [60]:
def transformer(list_sequence_train, list_sequence_val, train_y, val_y, num_features):
    
    with tf.variable_scope('data'):
    
        prob_input = tf.placeholder_with_default(1.0, shape=())
        prob_output = tf.placeholder_with_default(1.0, shape=())
        X = tf.placeholder(shape = [sequence_lenght, num_features], dtype = tf.float32, name = "input")
        X_or = tf.nn.dropout(X, prob_input)
        y = tf.placeholder(shape = [1,1], dtype = tf.float32, name = "label")
        multi_head_concat = tf.zeros((sequence_lenght, dim_transform1), tf.float32)
    
    for layer in range(1, layer_number+1):
        for head in range(1, head_number+1):

            multi_head_concat = multi_head_layer(X_or, layer, head, prob_output, 
                                                 multi_head_concat = multi_head_concat, num_features=num_features)

#         with tf.variable_scope('attention_head_'+ str(head)):

#             W_K = tf.Variable(tf.random_normal(shape = (num_features, dim_transform1), stddev = 0.1), name = "weights_keys", trainable = True)
#             W_Q = tf.Variable(tf.random_normal(shape = (num_features, dim_transform1), stddev = 0.1), name = "weights_queries", trainable = True)
#             W_V = tf.Variable(tf.random_normal(shape = (num_features, dim_transform1), stddev = 0.1), name = "weights_values", trainable = True)

#             queries = tf.matmul(X_or, W_K) # try with relu on these after that run finishes
#             keys = tf.matmul(X_or, W_Q)
#             values = tf.matmul(X_or, W_V)

#             output, attention_weights = self_attention(queries, keys, values)
#             output = tf.nn.dropout(output, prob_output, noise_shape=[1,dim_transform1])
#             #output = tf.add(values, output) # residual layer
#             #output = normalize_with_moments(output) # normalize residuals
#             multi_head_concat = tf.concat([multi_head_concat, output], axis = 1)
            
        multi_head_concat = multi_head_concat[:,dim_transform1:]
        dim_last = int(dim_transform1)
        W_concat = tf.Variable(tf.random_normal(shape = (dim_transform1*head_number, dim_last), stddev = 0.1), name = "weights_concat", trainable = True)
        X_or = tf.nn.relu(tf.matmul(multi_head_concat, W_concat))
        multi_head_concat = tf.zeros((sequence_lenght, dim_last), tf.float32)
    
    with tf.variable_scope('logits'):
    
        #multi_head_concat = multi_head_concat[:,dim_transform1:]
        # can do dropout here too
        #projected_dim = dim
        #W_concat = tf.Variable(tf.random_normal(shape = (dim_transform1*head_number, projected_dim), stddev = 0.1), name = "weights_concat", trainable = True)
        #multi_head_concat = tf.nn.relu(tf.matmul(multi_head_concat, W_concat))

        X_or = tf.reshape(tf.reshape(X_or, [-1]), shape = (sequence_lenght*dim_last,1)) #flatten the 2d array
        X_or = tf.nn.dropout(X_or, prob_output, noise_shape=[sequence_lenght*dim_last,1])
        W_final = tf.Variable(tf.random_normal(shape = (sequence_lenght*dim_last, 1), stddev = 0.1), name = "weights_final", trainable = True)
        pred_label = tf.nn.relu(tf.matmul(tf.transpose(X_or), W_final))
        
        
    with tf.variable_scope('loss'):
    
        loss =  tf.losses.mean_squared_error(predictions = pred_label, labels = y)
        
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(loss)

    print(tf.trainable_variables())
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

    val_loss_list, test_loss_list = [], []

    for i in range(1, n_epochs+1):

        train_pred, train_truth = [], []
        val_test_pred, val_test_truth = [], []
        indices = list(range(len(list_sequence_train)))
        shuffle(indices)

        for index in indices:
            X_batch = np.reshape(list_sequence_train[index], (sequence_lenght, num_features))
            Y_batch = np.reshape(train_y[index], (1,1))
            _, pred = sess.run([train_op, pred_label], feed_dict = {X: X_batch, y: Y_batch, 
                                                                    prob_input: dropout_input, prob_output: dropout_output})

            train_pred.append(pred)
            train_truth.append(Y_batch)

        for index in range(len(list_sequence_val)):
            # batch is size 1 here: stochastic gradient descent
            X_batch = np.reshape(list_sequence_val[index], (sequence_lenght, num_features))
            Y_batch = np.reshape(val_y[index], (1,1))

            pred_avg = 0
            for _ in range(num_test_runs):
                pred = sess.run([pred_label], feed_dict = {X: X_batch, y: Y_batch, prob_input: 1, prob_output: 1})
                pred_avg += pred[0]

            val_test_pred.append(pred_avg/num_test_runs)
            val_test_truth.append(Y_batch)

        if i == 1:
            # shuffle because the bags in the list are ranked (increasing order)
            range_arr = np.arange(len(val_test_truth))
            half_lenght = int(len(val_test_truth)/2)
            np.random.shuffle(range_arr)
            indices_val, indices_test = range_arr[:half_lenght], range_arr[half_lenght:]

        val_test_truth, val_test_pred = np.array(val_test_truth), np.array(val_test_pred)
        val_pred, val_truth = val_test_pred[indices_val], val_test_truth[indices_val]
        test_pred, test_truth = val_test_pred[indices_test], val_test_truth[indices_test]

        training_loss = compute_rmse(train_pred, train_truth)
        val_loss = compute_rmse(val_pred, val_truth)
        test_loss = compute_rmse(test_pred, test_truth)

        val_loss_list.append(val_loss)
        test_loss_list.append(test_loss)

        if (i == 1) or (i % 10 == 0):
            print("EPOCH " + str(i))
            print("TRAINING LOSS: " + str(training_loss))
            print("VAL LOSS: " + str(val_loss), "TEST LOSS: " + str(test_loss))

    min_index = get_min_index(np.array(val_loss_list))
    
    lowest_val_loss = val_loss_list[min_index]
    loss_test = test_loss_list[min_index]

    return lowest_val_loss, loss_test


In [ ]:
for dim in [24]:
    validation_loss_list, test_loss_list = [], []
    print("FOR DIM = " + str(dim))
    lowest_val_loss, loss_test_list = inference_attention(1, 5, False, 1)
    validation_loss_list.append(np.mean(lowest_val_loss))
    test_loss_list.append(np.mean(loss_test_list))
    print(np.mean(lowest_val_loss), np.mean(loss_test_list))

FOR DIM = 24
[<tf.Variable 'attention_head_1_1/weights_keys:0' shape=(12, 16) dtype=float32_ref>, <tf.Variable 'attention_head_1_1/weights_queries:0' shape=(12, 16) dtype=float32_ref>, <tf.Variable 'attention_head_1_1/weights_values:0' shape=(12, 16) dtype=float32_ref>, <tf.Variable 'weights_concat:0' shape=(16, 16) dtype=float32_ref>, <tf.Variable 'attention_head_2_1/weights_keys:0' shape=(16, 16) dtype=float32_ref>, <tf.Variable 'attention_head_2_1/weights_queries:0' shape=(16, 16) dtype=float32_ref>, <tf.Variable 'attention_head_2_1/weights_values:0' shape=(16, 16) dtype=float32_ref>, <tf.Variable 'weights_concat_1:0' shape=(16, 16) dtype=float32_ref>, <tf.Variable 'attention_head_3_1/weights_keys:0' shape=(16, 16) dtype=float32_ref>, <tf.Variable 'attention_head_3_1/weights_queries:0' shape=(16, 16) dtype=float32_ref>, <tf.Variable 'attention_head_3_1/weights_values:0' shape=(16, 16) dtype=float32_ref>, <tf.Variable 'weights_concat_2:0' shape=(16, 16) dtype=float32_ref>, <tf.Variab

In [55]:
tf.reset_default_graph()

In [113]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    print(shape)
    print(len(shape))
    variable_parameters = 1
    for dim in shape:
        print(dim)
        variable_parameters *= dim.value
    print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

(12, 128)
2
12
128
1536
(12, 128)
2
12
128
1536
(12, 128)
2
12
128
1536
(128, 12)
2
128
12
1536
(1200, 1)
2
1200
1
1200
7344
